# Parameters

In [303]:
list_of_currencies = ['MXN','JPY','INR','HKD','GBP','EUR','CNY','CHF','CAD','AUD']
lag = 3

In [304]:
import numpy as np
import pandas as pd

# Load data

In [305]:
import quandl as qd
qd.ApiConfig.api_key = "_4PBEc5xxKVoQoYxWcgG"

In [306]:
def get_currency(cur):
    return qd.get('CUR/'+cur,transformation="rdiff")

# Networks

In [307]:
from pgmpy.models import BayesianModel
import abc
import xml.etree.ElementTree as ET #a pretty well-documented xml module

In [308]:
class Basic:
    
    def __init__( self, params, start=0,lag=0):
        self.list_of_params = params
        self.time_frame = range(start,start+lag+1)
        self.model = BayesianModel()
        for t in self.time_frame:
            for s in params:
                self.model.add_node(Basic.get(s,t))
        
    @staticmethod
    def get(param, lag):
        return param+'_'+str(lag)
    
    def store_xml(self, name):
        node2pos = dict()
        node_space = 200
        for i in range(len(self.list_of_params)):
            for j in self.time_frame:
                node2pos[Basic.get(self.list_of_params[i],j)]=(i*node_space,(j-self.time_frame[0])*node_space)
        bif = ET.Element("BIF")
        network = ET.SubElement(bif, "NETWORK")
        bif.set ("VERSION","""0.3""")
        ET.SubElement(network, "NAME").text = "bayesiannetwork"
        for node in self.model.nodes():
            var = ET.SubElement(network,"VARIABLE")
            var.set("TYPE","""nature""")
            ET.SubElement(var,"NAME").text = node
            ET.SubElement(var,"OUTCOME").text = "increase"
            ET.SubElement(var,"OUTCOME").text = "decrease"
            ET.SubElement(var,"PROPERTY").text = "position =" + str(node2pos[node])
        for node in self.model.nodes():
            cpd = ET.SubElement(network,"DEFINITION")
            ET.SubElement(cpd,"FOR").text = node
            number_of_parents = len(self.model.get_parents(node))
            for par in self.model.get_parents(node):
                ET.SubElement(cpd,"GIVEN").text = par
            s = " "
            for i in range(2*2**number_of_parents):
                s+="0.5 "
            ET.SubElement(cpd,"TABLE").text = s
        tree = ET.ElementTree(bif)
        file_name = '{}.xml'.format(name)
        tree.write(file_name)
                
    def generate_table(self):
        self.table = pd.DataFrame()
        col_names = []
        for cur in self.list_of_params:
            data = get_currency(cur)
            data.RATE = list(map(lambda x: '1' if x>=0 else '0',data.RATE)) # 1 for "increase", 0 for "decrease"
            for i in range(self.time_frame[0]):
                data=data.shift(1)
            for l in self.time_frame:
                self.table=pd.concat([self.table,data],axis=1)
                data=data.shift(1)
                col_names.append(Basic.get(cur,l))
        self.table.columns = col_names
        
    def load_table(self,name):
        file_name = '{}.dat'.format(name)
        self.table = pd.read_csv( file_name, parse_dates=[0] )
        self.table.replace('N/A',np.nan,inplace=True)
        self.table.replace(1.0,'1',inplace=True)
        self.table.replace(0.0,'0',inplace=True)
        
    def train(self):
        self.model.fit(data=self.table,complete_samples_only=True)
    
    def query(self, target, observation={}):
        df = pd.DataFrame(list(observation.values()))
        df = df.T
        df.columns = list(observation.keys())
        print(df)
        probs = self.model.predict_probability(data=df)
        print(probs)
        ans = {}
        for var in target:
            state=0
            p = []
            while (True):
                col_name = Basic.get(var,state)
                print(col_name)
                if (col_name in probs.columns):
                    p.append(probs[col_name][0])
                else:
                    break
                state += 1
            ans[var]=p
        return ans

In [309]:
class BNGrid(Basic):
    def __init__( self, params,depth,start=0,lag=0):
        if (depth>lag):
            raise
        super().__init__(params,start,lag)
        for t in range(start,start+lag+1):
            for s in params:
                x = BNFull.get(s,t)
                for tt in range(t+1,min(start+lag+1,t+depth+1)):
                    for ss in params:
                        par = BNFull.get(ss,tt)
                        self.model.add_edge(par,x)

In [310]:
class BNFull(BNGrid):
    def __init__( self, params,start=0,lag=0):
        super().__init__(params,depth=1,start=start,lag=lag)

In [311]:
class BNCascade(BNFull):
    def __init__(self,params,lag,target):
        super().__init__(params,start=1,lag=lag-1)
        target = BNCascade.get(target,0)
        self.model.add_node(target)
        for x in self.model.nodes():
            if (x!=target):
                self.model.add_edge(x,target)

# Store 

In [312]:
def store_table(table,name):
    t = table.copy()
    t.replace(np.nan,'N/A',inplace=True)
    file_name = '{}.dat'.format(name)
    t.to_csv( file_name, index = False )

# Main

In [313]:
network = BNFull(params=list_of_currencies,lag=3)

In [314]:
#network.generate_table()
network.load_table('dataforsamiam') #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [315]:
network.store_xml('xmlforsamiam') 

In [316]:
#store_table(network.table,'dataforsamiam')

In [317]:
network.train()

In [ ]:
network.query(['EUR_0'],{'EUR_1':1,'EUR_2':0})

   EUR_1  EUR_2
0      1      0


In [ ]:
network.table